# Title

## Background

In 1950 the population of Detroit was 1,850,000, making it second only to Chicago in the Great Lakes Region of the United States.  It had a robust manufactuing sector which buttressed its position as one of America's leading cities and was considered to be a center of economic and cultural development.  Since then, for many reasons that this project will not specifically explore, Detroit has been in a steady state of decline.  In 2015 its population fell below 680,000, removing it from the top 20 cities in the United States for the first time in 165 years.  As of the 2020 Census, Detroit's population was 639,111, placing it in position 27. Some [research](https://www.milmi.org/_docs/publications/Population_Projections_2045.pdf) suggests that Detroit's population has reached a minimum and there is much effort being spent to revitilize the city in order to make it a more attractive place to live.

## Problem Statement

While the overarching problems in Detroit are well known it is not necessarily clear which of these problems are most holding Detroit back.  This project seeks to use county level data collected from every county in the United States to fit a machine learning model to these data and the Human Development Index (HDI) of the county.  This model will then be applied to the specific circumstances of Wayne County, Michigan (of which half of the land area and vast majority of the population are within the city limits of Detroit) in order to determine the top 5 areas of development that concerned individuals in Detorit could focus on to best increase the HDI of the area.

### Import Data

The data used for this analysis include economicand social data from [StatsAmerica](https://www.statsamerica.org/About.aspx), health data from [County Health Rankings & Roadmaps](https://www.countyhealthrankings.org/explore-health-rankings/use-data), and crime data from [The University of Michigan](https://www.icpsr.umich.edu/web/pages/).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


df = pd.read_csv('./data/metrics_for_development.csv')
life = pd.read_csv('./data/life_expectancy.csv')
income = pd.read_csv('./data/income.csv')
health = pd.read_csv('./data/health.csv')
crime = pd.read_csv('./data/crime.csv')
social = pd.read_csv('./data/social_context.csv')
df

,IBRC_Geo_ID,Statefips,Countyfips,Description,Year,M4D_Code,Code Description,M4D_Data
0,1001,1,1,"Autauga County, AL",2019,100,Headline M4D Index,0.666112
1,1001,1,1,"Autauga County, AL",2019,1000,Full-Time Work,0.661573
2,1001,1,1,"Autauga County, AL",2019,10000,Grocery stores per capita,0.072548
3,1001,1,1,"Autauga County, AL",2019,10100,Farmers' markets per capita,0.018166
4,1001,1,1,"Autauga County, AL",2019,10200,SNAP benefits per capita,17.905272
...,...,...,...,...,...,...,...,...
708847,56045,56,45,"Weston County, WY",2019,9500,Years of potential life lost to premature deat...,6921.612793
708848,56045,56,45,"Weston County, WY",2019,9600,Population that has low access to grocery stores,1237.267037
708849,56045,56,45,"Weston County, WY",2019,9700,Percent of population that has low access to g...,17.299595
708850,56045,56,45,"Weston County, WY",2019,9800,Low-income population with low access to groce...,302.697580


The core data is in long format, so a pivot table is constructed to make every county in the United States its own record with the features collected for each county placed in columns.

In [2]:
df.drop(columns = ['IBRC_Geo_ID', 'Statefips', 'Countyfips', 'Year', 'M4D_Code'], inplace = True)

df = df.pivot_table(index = 'Description', columns = 'Code Description', values = 'M4D_Data')

df.reset_index(inplace = True)
df.head()

Code Description,Description,2010 population,2016 population estimate,Adjusted gross income reported on itemized returns,"Average earnings per worker growth, 2001-2016",Average of sk_litp and sk_lit8p,Capital outlays per pupil,Charitable Giving and Civil Society,Civilian population aged 25 and up,Commuting to Work,...,"Violent crime events per 1,000 population",Worked Outside County of Residence,Worked in County of Residence,Years of potential life lost to premature death (age-adjusted),"emp. growth, 2001-2016","emp. in arts occupations, 2007-11 average","emp. in creative class occupations, 2007-11 average","emp. in social assistance and outreach industries per 10,000 population",emp. in social assistance industries (NAICS 624),"emp., 2007-11 average"
0,"Abbeville County, SC",25417.0,24951.0,317613.0,0.014772,0.894183,0.215969,0.569973,17077.0,0.172516,...,3.424064,5117.0,4222.0,9540.950195,-0.017176,30.0,1220.0,34.868342,87.0,9815.0
1,"Acadia Parish, LA",61773.0,62372.0,904289.0,0.029137,0.875001,0.340423,0.301410,40069.0,0.211959,...,4.305457,12425.0,12027.0,10226.620120,-0.000018,85.0,4210.0,73.109729,456.0,24975.0
2,"Accomack County, VA",33164.0,33060.0,672436.0,0.038428,0.879827,0.153947,0.381361,23850.0,0.234122,...,2.473157,2943.0,11199.0,9538.187500,-0.004523,90.0,2605.0,120.992136,400.0,14880.0
3,"Ada County, ID",392365.0,425798.0,18171426.0,0.023481,0.961839,0.391907,0.301511,281818.0,0.157836,...,2.277567,15226.0,189943.0,4919.320313,0.013088,2240.0,58125.0,112.424201,4787.0,188250.0
4,"Adair County, IA",7682.0,7330.0,119224.0,0.032786,0.950666,1.954139,0.322050,5275.0,0.204575,...,0.672676,1577.0,2082.0,8897.926758,-0.003617,15.0,630.0,140.518417,103.0,3970.0


The geographic location data is located in the ```Description``` column.  For merging activities performed in the next steps these are seperated into ```state``` and ```county``` columns.

In [3]:
df['state'] = df['Description'].apply(lambda s: s[-2:])
df['county'] = df['Description'].apply(lambda s: s[:-4])
df['county'] = df['county'].apply(lambda s: s.replace(' County', ''))
df['county'] = df['county'].apply(lambda s: s.replace(' Parish', ''))
df['county'] = df['county'].apply(lambda s: s.lower())

### Income Data

The income data uses a long format for the state identifier, so a map is created to convert this to the two-letter state code.

In [4]:
income.head()

,county,state_long,per capita income,median household income,median family income,population,number of households
0,Shelby,Alabama,"$33,313","$68,770","$82,558","198,366","74,144"
1,Madison,Alabama,"$31,933","$58,434","$75,766","339,279","132,975"
2,Jefferson,Alabama,"$26,906","$45,429","$59,095","658,552","259,634"
3,Baldwin,Alabama,"$26,766","$50,221","$60,864","187,114","73,283"
4,Limestone,Alabama,"$25,020","$48,619","$57,858","85,264","31,794"


In [5]:
abrv_map = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

state_map = dict((y, x) for x, y in abrv_map.items())

In [6]:
income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3197 entries, 0 to 3196
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   county                   3146 non-null   object
 1   state_long               3197 non-null   object
 2   per capita income        3197 non-null   object
 3   median household income  3197 non-null   object
 4   median family income     3197 non-null   object
 5   population               3197 non-null   object
 6   number of households     3197 non-null   object
dtypes: object(7)
memory usage: 175.0+ KB


Most of the data from this data set is either replicated in the core data or is extremely highly correlated, so only the ```per capita income``` is retained.

In [7]:
income['state'] = income['state_long'].map(state_map)
income = income[['county', 'state', 'per capita income']]
income.dropna(inplace = True)
income['county'] = income['county'].apply(lambda s: s.lower())

df = pd.merge(df, income, how = 'inner', left_on = ['state', 'county'], right_on = ['state', 'county'])
df.shape

(2977, 231)

### Life Expectancy Data

In [8]:
life.head()

,State,County,"Male life expectancy, 1985 (years)","Female life expectancy, 1985 (years)","Male life expectancy, 1990 (years)","Female life expectancy, 1990 (years)","Male life expectancy, 1995 (years)","Female life expectancy, 1995 (years)","Male life expectancy, 2000 (years)","Female life expectancy, 2000 (years)","Male life expectancy, 2005 (years)","Female life expectancy, 2005 (years)","Male life expectancy, 2010 (years)","Female life expectancy, 2010 (years)","Difference in male life expectancy, 1985-2010 (years)","Difference in female life expectancy, 1985-2010 (years)"
0,United States,NaN,70.8,77.8,71.5,78.3,72.5,78.8,74.1,79.2,74.9,79.9,76.1,80.8,5.2,3.0
1,Alabama,NaN,69.1,76.9,69.4,77.1,69.9,77.2,71.0,77.2,71.5,77.3,72.4,77.8,3.3,0.9
2,Alabama,Autauga,68.1,77.0,68.5,77.0,69.3,77.3,71.7,77.6,72.5,77.4,73.3,78.8,5.2,1.8
3,Alabama,Baldwin,71.1,78.8,71.9,79.2,72.4,79.3,73.9,79.2,73.8,79.7,75.0,80.3,3.8,1.5
4,Alabama,Barbour,66.8,76.0,66.7,76.7,68.4,76.9,70.0,76.2,70.5,77.0,72.2,77.2,5.3,1.1


Here we aggregate the male and female life expectancy into a single parameter based on the assumption that males and females are each nearly 50% of the population.

In [9]:
life['life expectancy'] = (life['Male life expectancy, 2010 (years)'] + life['Female life expectancy, 2010 (years)']) / 2

The ```state_map``` is again applied to make this data fit the merge format.

In [10]:
life['State'] = life['State'].map(state_map)
life = life[['State', 'County', 'life expectancy']]
life['County'] = life.dropna()['County'].apply(lambda s: s.lower())
life.rename(columns = {'County' : 'county', 'State': 'state'}, inplace = True)

In [11]:
df = pd.merge(df, life, how = 'inner', left_on = ['state', 'county'], right_on = ['state', 'county'])

### Social Context Data

The social context data is also in long format, so a similar procedure as used above is applied to convert the data into a format suitable to joining to the base data set.

In [12]:
social.head()

,IBRC_Geo_ID,Statefips,Countyfips,Description,Year,Social_Context_Code,Social_Context_Code_Description,Social_Context_Domain_Data,Time_Period
0,10001,10,1,"Kent County, DE",2019,211,Hopefulness,87.332678,1972-2017
1,10001,10,1,"Kent County, DE",2019,104,Employment Rate,95.300000,2017
2,10001,10,1,"Kent County, DE",2019,103,Income Per Capita,26118.000000,2017
3,10001,10,1,"Kent County, DE",2019,102,Income Mobility,-0.441463,2017
4,10001,10,1,"Kent County, DE",2019,101,Entrepreneurship,10.732984,2013


In [13]:
social = social.pivot_table(index = 'Description', columns = 'Social_Context_Code_Description', values = 'Social_Context_Domain_Data')

In [14]:
social.head()

Social_Context_Code_Description,Agreeableness,Belief In Science,Collectivism,Conflict Awareness,Conscientiousness,Empathy,Employment Rate,Entrepreneurship,Extraversion,Gender Equality,Hopefulness,Income Mobility,Income Per Capita,Neuroticism,Openness,Religiosity,Risk Taking,Selflessness,Tolerance,Work Ethic
Description,,,,,,,,,,,,,,,,,,,,
"Abbeville County, SC",87.350707,65.893083,67.059286,63.598034,84.252438,78.089447,94.7,7.692308,87.276108,67.778351,58.501424,-0.283619,18460.0,78.491310,77.648742,71.075626,69.614094,73.141394,67.729606,71.832059
"Acadia Parish, LA",85.748276,68.113642,71.956409,47.259230,82.910971,80.295487,92.6,9.189189,86.121050,67.075197,83.106981,0.211839,20887.0,81.690957,76.729678,71.387831,68.553784,78.489674,69.965661,68.410313
"Accomack County, VA",87.703673,67.760489,68.407928,59.665657,86.272667,80.891043,95.2,9.279778,84.909876,69.864440,85.275714,-0.376874,23337.0,73.643432,82.225834,74.263923,79.254893,78.295705,70.116249,67.770044
"Ada County, ID",85.328719,68.628364,65.277778,59.476135,83.822607,84.689542,96.7,15.795455,86.641503,62.638412,87.717745,0.178408,30086.0,77.511744,80.619908,63.886517,88.000000,70.917563,78.131808,74.159420
"Adair County, IA",84.314998,69.132837,67.194144,61.673888,80.760009,77.533570,97.0,8.376963,88.722187,68.941854,78.545125,1.447790,27358.0,80.143343,78.037608,66.430970,77.277567,72.782303,65.402275,70.372691


In [15]:
social.drop(columns = 'Income Per Capita', inplace = True)
social.reset_index(inplace = True)
social['state'] = social['Description'].apply(lambda s: s[-2:])
social['county'] = social['Description'].apply(lambda s: s[0:s.find(',')])
social['county'] = social['county'].apply(lambda s: s.replace(' County', ''))
social['county'] = social['county'].apply(lambda s: s.replace(' Parish', ''))
social['county'] = social['county'].apply(lambda s: s.lower())

In [16]:
social.drop(columns = ['Description'], inplace = True)
df = pd.merge(df, social, how = 'inner', left_on = ['state', 'county'], right_on = ['state', 'county'])

### Crime Data

Again, similar procedures are used to transform the crime data into the correct shape and format.

In [17]:
crime.head()

,county_name,crime rate per 100000,Murder rate per 100000,Rape rate per 100000,Robbery rate per 100000,Aggrevated Assault rate per 100000,Burglery rate per 100000,Larceny rate per 100000,Motor Vehicle Theft rate per 100000,Arson rate per 100000
0,"St. Louis city, MO",1791.995377,37.372494,62.810914,558.389026,1133.422944,1568.702578,4331.126577,1112.695342,145.721320
1,"Crittenden County, AR",1754.914968,16.081695,76.388051,331.684960,1330.760262,2979.134001,3523.901419,379.930045,56.285933
2,"Alexander County, IL",1664.700485,13.107878,26.215756,65.539389,1559.837462,1074.845982,2411.849522,157.294534,26.215756
3,"Kenedy County, TX",1456.310680,0.000000,728.155340,242.718447,485.436893,1213.592233,970.873786,970.873786,0.000000
4,"De Soto Parish, LA",1447.402430,11.077059,14.769413,62.770003,1358.785954,550.160617,1824.022450,221.541188,0.000000


In [18]:
crime['state'] = crime['county_name'].apply(lambda s: s[-2:])
crime['county'] = crime['county_name'].apply(lambda s: s[0:s.find(',')])
crime['county'] = crime['county'].apply(lambda s: s.replace(' County', ''))
crime['county'] = crime['county'].apply(lambda s: s.replace(' Parish', ''))
crime['county'] = crime['county'].apply(lambda s: s.lower())

In [19]:
df = pd.merge(df, crime, how = 'inner', left_on = ['state', 'county'], right_on = ['state', 'county'])

### Health Data

The health data includes some non-numeric data which needs to be converted.  In particular, the health care professionals per capita are presented in a ratio format of x:y, where y is one provider and x is the number of people per provider.  The following cell extracts an integer from this format and also imputes data to the mean where no data is available.

In [20]:
health['Presence of Water Violation'] =health['Presence of Water Violation'].apply(lambda n: 1 if n == 'Yes' else 0)

for feature in ['Dentist Ratio', 'Primary Care Physicians Ratio','Mental Health Provider Ratio']:
    health[feature].fillna('999999:0', inplace = True)
    health[feature] = health[feature].apply(lambda n: int(n[0:n.find(':')]))
    impute = health[health[feature] != 999999][feature].mean()
    health[feature] = health[feature].apply(lambda n: impute if n == 999999 else n)

for feature in health.drop(columns = ['State', 'County']):
    impute = health[feature].mean()
    health[feature] = health[feature].apply(lambda n: impute if n == np.NaN else n)

In [21]:
health['State'] = health['State'].map(state_map)
health.dropna(inplace = True)
health['County'] = health['County'].apply(lambda s: s.replace(' County', '').lower())
health['County'] = health['County'].apply(lambda s: s.replace(' Parish', '').lower())
health.rename(columns = {'County' : 'county', 'State': 'state'}, inplace = True)

In [22]:
health[health['state'] == 'LA']

,state,county,% Fair or Poor Health,% Low birthweight,% Smokers,Food Environment Index,% Physically Inactive,% With Access to Exercise Opportunities,% Excessive Drinking,% Driving Deaths with Alcohol Involvement,...,% Children in Single-Parent Households,Social Association Rate,Annual Average Violent Crimes,Violent Crime Rate,Injury Death Rate,Average Daily Air Polution above 2.5 ppm,Presence of Water Violation,% Severe Housing Problems,% Drive Alone to Work,% Long Commute - Drives Alone
1132,LA,acadia,25,9.0,27,7.3,32,43.0,19,25.0,...,29.0,7.9,274.0,443.0,83.0,8.7,1,13,85,38
1133,LA,allen,24,9.0,27,6.7,33,73.0,20,14.0,...,21.0,11.3,27.0,124.0,95.0,8.5,1,12,86,40
1134,LA,ascension,18,9.0,20,7.9,25,73.0,20,36.0,...,24.0,5.2,401.0,337.0,67.0,10.1,1,11,86,50
1135,LA,assumption,25,11.0,27,6.3,28,35.0,20,24.0,...,47.0,4.9,73.0,319.0,84.0,9.0,1,11,87,51
1136,LA,avoyelles,28,12.0,28,6.4,36,24.0,20,41.0,...,38.0,8.4,227.0,584.0,97.0,8.8,1,14,84,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,LA,webster,27,12.0,27,5.9,32,51.0,18,34.0,...,38.0,12.4,143.0,368.0,81.0,9.5,1,15,88,38
1192,LA,west baton rouge,21,11.0,22,7.6,27,74.0,21,41.0,...,28.0,6.1,89.0,351.0,93.0,9.0,0,12,87,33
1193,LA,west carroll,26,11.0,28,7.0,32,33.0,20,13.0,...,37.0,18.2,46.0,402.0,111.0,8.4,1,8,86,30
1194,LA,west feliciana,21,9.0,22,6.5,23,15.0,22,33.0,...,42.0,5.2,40.0,261.0,99.0,8.9,0,10,89,44


In [23]:
df = pd.merge(df, health, how = 'left', left_on = ['state', 'county'], right_on = ['state', 'county'])

### Cleaning Combined Dataframe

The health care provider data was imputed in the previous step; however, there is missing data here and there throughout the dataset.  The following cell imputes the mean into all null cells.

In [24]:
for feature in list(df.select_dtypes(include=[np.number]).columns.values):
    impute = df[feature].mean()
    df[feature].fillna(impute, inplace = True)

The data includes a number of gross values where the data represents values that scale with the gross population of the county.  In these cases the results are all standardized to per capita values.

In [25]:
non_pop_adjusted = ['Adjusted gross income reported on itemized returns', 
                    'Low-income population with low access to grocery stores',
                    'Number of persons who moved in/out of county in last year from/to any US county/territory',
                    'Number of tax returns, 2015',
                    'Population that has low access to grocery stores',
                    'Reported instances of property crime',
                    'Reported instances of violent crime  ',
                    'Total civilian population aged 18-64',
                    'Total civilian population aged 25-34',
                    'Total civilian population, 2016',
                    'Total fall 2015 public school enrollment',
                    'Total school revenue from federal sources, thousands of current dollars',
                    'Total school revenue from local sources, thousands of current dollars',
                    'Total school revenue from state sources, thousands of current dollars',
                    'Total school revenue, thousands of current dollars',
                    'Total workers 16+ commuting to work, 2016',
                    'Worked Outside County of Residence  ',
                    'Worked in County of Residence  ',
                    'Years of potential life lost to premature death (age-adjusted)',
                    'emp. in arts occupations, 2007-11 average',
                    'emp. in creative class occupations, 2007-11 average',
                    'emp. in social assistance industries (NAICS 624)',
                    'emp., 2007-11 average']

for feature in non_pop_adjusted:
    df[feature] = df[feature] / df['2016 population estimate']
    df.rename(columns = {feature : feature + ' rate'}, inplace = True)

#### Calculate Indices

We were not able to find county level data on the HDI.  However, this value can be caluclated form the data that is available.  Note that as calculations are made various fields are added to the ```drop_list``` because these fields will be almost perfectly correlated to the calculated HDI since they are the parameters used in the calculation of it and will, therefore, leak data into the final model.  At the end of the data cleaning process the ```drop_list``` will be used to remove these highly correlated features.

In [26]:
df['per capita income'] = df['per capita income'].apply(lambda x: int(x.strip('$').replace(',','')))

One of the components of the HDI is the number of years of education that the average person completes.  The United States does not collect this data specifically, rather it collects data on the percentage of people who successfully reach certain educational milestones.  The following code collects this data into a single number by first multiplying the percent of people who complete certain milestones by the number of years of education that that milestone represents.  Then this total sum is divided by the total population to calculate ```MYS``` which is the mean number of years of education completed.

In [27]:
years_of_edu = [0, 4, 6, 8, 12, 16]
drop_list = [
'Population over the age of 25 that completed no school',
'Population over the age of 25 that completed nursery to 4th grade',
'Population over the age of 25 that completed 5th and 6th grade',
'Population over the age of 25 that completed 7th and 8th grade',
'Percent of population with high school diploma or equivalent',
"Percent of population with at least a bachelor's degree"
]

for i, item in enumerate(drop_list):
    if i == 0:
        gross_years_edu = df[item] * years_of_edu[i]
    elif i < 4:
        gross_years_edu = gross_years_edu + (df[item] * years_of_edu[i])
    else:
        gross_years_edu = gross_years_edu + ((df[item] * years_of_edu[i] * df['2016 population estimate']) / 100)

df['years edu'] = gross_years_edu
drop_list.append('years edu')
df['MYS'] = df['years edu'] / df['2016 population estimate']

drop_list.append('Percent of the population 25 and above that have completed more than 8th grade')

The Educational Index ```EI``` is a parameter that combines both the ```MYS``` and a parameter that represents the number of years of school that the average two-year-old today will be *expected* to complete.  Note that this *expected education* is not a parameter that is captured at a county level basis in the United States, so the value of [16.5](https://ourworldindata.org/grapher/expected-years-of-schooling) for the *country as a whole* is used here. Note that this will skew the HDI in high educational attainment counties down and vice versa. 

The life expectancy index ```LE``` is much simpler to calculate as this data is readily available at both the county and zio code level.  Similarly, the Internal Revenue Service mainatins high quality data about per capita income to calculate the income index ```II```.  

All three of these "sub-indices" are combined according to the [formula](https://en.wikipedia.org/wiki/Human_Development_Index) developed by the United Nations Development Programme.



In [28]:
df['EI'] = ((df['MYS'] / 15) + (16.5 / 18)) / 2

df['LE'] = (df['life expectancy'] - 20 ) / 65
drop_list.append('life expectancy')

df['II'] = (np.log(df['per capita income']) - np.log(100)) / (np.log(75_000) - np.log(100))
drop_list.append('per capita income')

df['HDI'] = (df['EI'] * df ['II'] * df['LE']) ** (1/3)

df['HDI'].mean()

0.854317219514784

The standard deviation of the HDI between counties in the United States is approximately 0.033.  While this seems like a small number, for context, two standard deviations separate the country with highest HDI (Norway) from the country in 39th place (The United Arab Emirates).

In [29]:
df['HDI'].std()

0.0331035571032904

The ```drop_list``` is further appended with some indices calculated in core dataset which are aggregates of various other features.  As a reminder, the purpose of this project is to determine which *specific* areas of improvement Detorit could focus on to best increase its HDI.  So, composite indicies (other than the target of HDI) are removed to allow the final analysis to see more granular results.

In [30]:
drop_list.extend(['Itemized contributions', 'QCEW emp., 2001-2016 Average  ', 'QCEW emp., 2016  ', 'county_name',
                  'Food Access', 'School Funding', 'Commuting to Work', 'Health', 'Industry Mix', 
                  'Creative Class Occupations and Creative Industries', 'Natural Amenities', 
                  'Charitable Giving and Civil Society', 'Full-Time Work', 'Crime', 
                  'Jobs, Earnings, and Productivity', 'Population Dynamics',
                  'Literacy and Education', 'Average of sk_litp and sk_lit8p'])

for item in df.columns:
    if 'index' in item.lower():
        drop_list.append(item)

# Use the set object to remove any duplicate items in the drop list
drop_list = list(set(drop_list))

df.drop(columns = drop_list, inplace = True)

### Final Inspection

Ultimately we end up with 194 features accross 2,969 coounties in the United States to pass on to the model.

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2969 entries, 0 to 2968
Columns: 194 entries, Description to HDI
dtypes: float64(191), object(3)
memory usage: 4.4+ MB


In [32]:
df.to_csv('./data/clean_data.csv', index = False)